In [3]:
import dotenv
import os
import openai
import json
import os
import pandas as pd

# .env 파일 로드 (있다면)
dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

In [13]:
df = pd.read_csv("data_detail.csv")
texts = df['text'].to_list()
contents = df['detail'].to_list()

In [29]:
client = openai.OpenAI()

# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """
당신은 소프트웨어 프로젝트의 요구사항 분석 전문가입니다.
주어진 프로젝트 개요를 바탕으로 현실적이고 구체적인 기능 요구사항과 성능 요구사항을 생성하는 것이 목표입니다.

생성 규칙:
1. 기능 요구사항(FUNCTIONAL): 사용자 관점에서 시스템이 제공해야 하는 구체적인 기능
2. 성능 요구사항(PERFORMANCE): 시스템의 응답시간, 처리량, 가용성, 확장성 등 측정 가능한 수치 포함
3. 각 요구사항은 명확하고 구현 가능하며 테스트 가능해야 함
4. 프로젝트 규모와 성격에 적합한 현실적인 수준이어야 함
5. 반드시 지정된 JSON 형식으로만 응답할 것
"""

# 파인튜닝된 모델 ID
MODEL_ID = 'gpt-4o-mini'

def detail_plus(project_overview,existing_requirements, additional_count = 5):
  # 구조화된 요청 프롬프트 (f-string 사용하지 않음)
    enhanced_prompt = f"""

    프로젝트 주제 :
    {project_overview}

    기존 요구사항 목록:
    {existing_requirements}

    위 기존 주제와 요구사항들을 참고하여 추가 요구사항을 생성해주세요
    또한 이렇게 생성된 요구사항은 원래의 '기존 요구사항 목록'에 추가시켜서 정렬 후 출력하세요:

    **생성 조건:**
    - 기존 주제와 요구사항과 일관성을 유지하면서 새로운 {additional_count}개의 요구사항 생성
    - 기존 주제와 요구사항의 도메인과 맥락을 고려
    - 기능 주제와 요구사항과 성능 요구사항을 적절히 혼합
    - 기존 요구사항과 중복되지 않는 새로운 관점의 요구사항

    **중요: 반드시 아래 JSON 형식으로만 응답하세요. 다른 텍스트, 설명, 주석은 절대 포함하지 마세요.**

    JSON 형식:
    [
      {{"requirementType": "FUNCTIONAL1", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "FUNCTIONAL2", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "FUNCTIONAL3", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "PERFORMANCE2", "content": "구체적인 성능 요구사항 (수치 포함)"}},
      {{"requirementType": "PERFORMANCE1", "content": "구체적인 성능 요구사항 (수치 포함)"}}
    ]

    응답은 반드시 위 JSON 배열로만 시작하고 끝나야 합니다.

    
    """
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"오류 발생: {e}"

In [ ]:
detail_lst = []
for text, content in zip(texts, contents) :
  result = json.loads(detail_plus(text, content))
  detail_lst.append(result)

df['detail'] = detail_lst 